In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import theano
from scipy.stats import norm, invgamma
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import logging
logger = logging.getLogger("pymc3")
logger.setLevel(logging.INFO)
logger = logging.getLogger("theano")
logger.setLevel(logging.ERROR)

np.random.seed(12345)

### Generate Ideal Data

In [ ]:
n_days = 400
n_teams = 32
gpd = 8

In [ ]:
lv_df = pd.read_csv('results/lv_df.csv')
games = pd.read_csv('results/games.csv')

### Model 1: Daily Updates, No Deltas

In [ ]:
def get_m1_posteriors(trace):
    posteriors = {}
    h_μ, h_σ = norm.fit(trace['h'])
    posteriors['h'] = [h_μ, h_σ]
    i_μ, i_σ = norm.fit(trace['i'])
    posteriors['i'] = [i_μ, i_σ]
    o_μ = []
    o_σ = []
    d_μ = []
    d_σ = []
    for i in range(n_teams):
        oᵢ_μ, oᵢ_σ = norm.fit(trace['o'][:,i])
        o_μ.append(oᵢ_μ)
        o_σ.append(oᵢ_σ)
        dᵢ_μ, dᵢ_σ = norm.fit(trace['d'][:,i])
        d_μ.append(dᵢ_μ)
        d_σ.append(dᵢ_σ)
    posteriors['o'] = [np.array(o_μ), np.array(o_σ)]
    posteriors['d'] = [np.array(d_μ), np.array(d_σ)]
    
    # Unified o and d variances
    #o_σ_α, _, o_σ_β = invgamma.fit(trace['o_σ'])
    #posteriors['o_σ'] = [o_σ_α, o_σ_β]
    #d_σ_α, _, d_σ_β = invgamma.fit(trace['d_σ'])
    #posteriors['d_σ'] = [d_σ_α, d_σ_β]
    return posteriors

def fatten_priors(prev_posteriors, init_priors, ratio):
    priors = prev_posteriors.copy()
    priors['h'][1] = np.minimum(priors['h'][1] * ratio, init_priors['h'][1] * ratio)
    priors['i'][1] = np.minimum(priors['i'][1] * ratio, init_priors['i'][1] * ratio)
    priors['o'][1] = np.minimum(priors['o'][1] * ratio, init_priors['o'][1] * ratio)
    priors['d'][1] = np.minimum(priors['d'][1] * ratio, init_priors['d'][1] * ratio)
    #priors['o_σ'][1] = priors['o_σ'][1] * ratio
    #priors['d_σ'][1] = priors['d_σ'][1] * ratio
    return priors

def m1_iteration(obs_data, priors):
    idₕ = obs_data['idₕ'].to_numpy()
    sₕ_obs = obs_data['sₕ'].to_numpy()
    idₐ = obs_data['idₐ'].to_numpy()
    sₐ_obs = obs_data['sₐ'].to_numpy()
    hw_obs = obs_data['hw'].to_numpy()
    
    with pm.Model() as model:
        # Global model parameters
        h = pm.Normal('h', mu=priors['h'][0], sigma=priors['h'][1])
        i = pm.Normal('i', mu=priors['i'][0], sigma=priors['i'][1])

        # Team-specific poisson model parameters
        o_star = pm.Normal('o_star', mu=priors['o'][0], sigma=priors['o'][1], shape=n_teams)
        d_star = pm.Normal('d_star', mu=priors['d'][0], sigma=priors['d'][1], shape=n_teams)
        o = pm.Deterministic('o', o_star - tt.mean(o_star))
        d = pm.Deterministic('d', d_star - tt.mean(d_star))
        λₕ = tt.exp(i + h + o[idₕ] - d[idₐ])
        λₐ = tt.exp(i + o[idₐ] - d[idₕ])

        # OT/SO home win bernoulli model parameter
        # P(T < Y), where T ~ a, Y ~ b: a/(a + b)
        pₕ = λₕ/(λₕ + λₐ)
        
        # Likelihood of observed data
        sₕ = pm.Poisson('sₕ', mu=λₕ, observed=sₕ_obs)
        sₐ = pm.Poisson('sₐ', mu=λₐ, observed=sₐ_obs)
        hw = pm.Bernoulli('hw', p=pₕ, observed=hw_obs)

        trace = pm.sample(500, tune=500, cores=2, progressbar=True)
        
        posteriors = get_m1_posteriors(trace)
        return posteriors

In [ ]:
start_day = 170

In [ ]:
starting_priors = pickle.load(open('results/starting_priors.pkl', 'rb'))

In [ ]:
window_sizes = [1] #[30, 60, 90]
fattening_factors = [1.5] #, 1.001, 1.01]

for ws in window_sizes:
    for f in fattening_factors:
        print('ws:{} and f:{}'.format(ws, f))
        priors = starting_priors.copy()
        iv1_rows = []
        for t in tqdm(range(start_day, n_days+1)):
            obs_data = games[((games['day'] <= t) & (games['day'] > (t - ws)))]
            posteriors = m1_iteration(obs_data, priors);
            iv_row = posteriors['h'] + posteriors['i'] + list(posteriors['o'][0]) + list(posteriors['o'][1]) + \
                     list(posteriors['d'][0]) + list(posteriors['d'][1])
            iv1_rows.append(iv_row)
            priors = fatten_priors(posteriors, starting_priors, f)
        
        col_names = ['h_μ', 'h_σ', 'i_μ', 'i_σ'] + ['o{}_μ'.format(i) for i in range(n_teams)] + \
                    ['o{}_σ'.format(i) for i in range(n_teams)] + ['d{}_μ'.format(i) for i in range(n_teams)] + \
                    ['d{}_σ'.format(i) for i in range(n_teams)]
        iv1_df = pd.DataFrame(iv1_rows, columns=col_names)
        iv1_df['day'] = list(range(start_day, start_day+len(iv1_rows)))
        iv1_df.to_csv('results/m1_{}d_f{}_iv_df.csv'.format(ws, f))

In [ ]:
col_names = ['h_μ', 'h_σ', 'i_μ', 'i_σ'] + ['o{}_μ'.format(i) for i in range(n_teams)] + \
            ['o{}_σ'.format(i) for i in range(n_teams)] + ['d{}_μ'.format(i) for i in range(n_teams)] + \
            ['d{}_σ'.format(i) for i in range(n_teams)]

In [ ]:
def plot_parameter_estimate(param):
    plt.figure(figsize=(10, 6))
    plt.title('Estimates for: ' + param)
    plt.plot(lv_df['day'], lv_df[param], color='blue')
    plt.plot(iv1_df['day'], iv1_df[param+'_μ'], color='red')
    upper1sd = iv1_df[param+'_μ'] + iv1_df[param+'_σ']
    lower1sd = iv1_df[param+'_μ'] - iv1_df[param+'_σ']
    upper2sd = iv1_df[param+'_μ'] + 2 * iv1_df[param+'_σ']
    lower2sd = iv1_df[param+'_μ'] - 2 * iv1_df[param+'_σ']
    plt.fill_between(iv1_df['day'], upper2sd, lower2sd, color='red', alpha=0.2)
    plt.fill_between(iv1_df['day'], upper1sd, lower1sd, color='red', alpha=0.2)
    plt.show()

In [ ]:
def plot_multi_parameter_estimate(param_list, y_lim=(-0.6, 0.6), grid_lines=0.10):
    imgsize = 4
    figsize = (15,15)
    rows = int(np.ceil(np.sqrt(len(param_list))))
    fig, axs = plt.subplots(rows, rows, figsize=figsize)
    ax = axs.flatten()
    for i in range(len(param_list)):
        param = param_list[i]
        ax[i].set_title('Estimates for: ' + param)
        ax[i].plot(lv_df['day'], lv_df[param], color='blue')
        ax[i].plot(iv1_df['day'], iv1_df[param+'_μ'], color='red')
        upper1sd = iv1_df[param+'_μ'] + iv1_df[param+'_σ']
        lower1sd = iv1_df[param+'_μ'] - iv1_df[param+'_σ']
        upper2sd = iv1_df[param+'_μ'] + 2 * iv1_df[param+'_σ']
        lower2sd = iv1_df[param+'_μ'] - 2 * iv1_df[param+'_σ']
        ax[i].fill_between(iv1_df['day'], upper2sd, lower2sd, color='red', alpha=0.2)
        ax[i].fill_between(iv1_df['day'], upper1sd, lower1sd, color='red', alpha=0.2)
        for y in np.arange(y_lim[0] + grid_lines, y_lim[1], grid_lines):
            ax[i].hlines(y, 1, n_days, colors='k', linestyles='dotted', alpha=0.4)
        ax[i].set_ylim(y_lim[0], y_lim[1])
        
    fig.tight_layout()
    plt.show()

In [ ]:
plot_parameter_estimate('i')

In [ ]:
plot_parameter_estimate('h')

In [ ]:
plot_multi_parameter_estimate(['o{}'.format(i) for i in range(32)])

In [ ]:
plot_multi_parameter_estimate(['d{}'.format(i) for i in range(32)])

In [ ]:
plot_parameter_estimate('o4')

In [ ]:
plot_parameter_estimate('o19')

In [ ]:
plot_parameter_estimate('d10')

In [ ]:
plot_parameter_estimate('d2')